# Example: Computing on the Dynex Platform with Python - NAE3SAT

Dynex is the world’s first neuromorphic supercomputing blockchain based on the DynexSolve chip algorithm,
a Proof-of-Useful-Work (PoUW) approach to solving real-world problems. This example demonstrates how to use the Dynex SDK to use Pyton to compute on the Dynex Platform with Python.

In [1]:
import dynex
import dimod

## Building a Binary Quadratic Model 

Binary quadratic models (BQMs) are problems of the form:

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>E</mi>
  <mo stretchy="false">(</mo>
  <mrow data-mjx-texclass="ORD">
    <mi mathvariant="bold">v</mi>
  </mrow>
  <mo mathvariant="bold" stretchy="false">)</mo>
  <mo mathvariant="bold">=</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">a</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">&lt;</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">b</mi>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">,</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">j</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <mi mathvariant="bold">c</mi>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo>&#x2208;</mo>
  <mo fence="false" stretchy="false">{</mo>
  <mo mathvariant="bold">&#x2212;</mo>
  <mn mathvariant="bold">1</mn>
  <mo mathvariant="bold">,</mo>
  <mo mathvariant="bold">+</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
  <mtext mathvariant="bold">&#xA0;or&#xA0;</mtext>
  <mo fence="false" stretchy="false">{</mo>
  <mn mathvariant="bold">0</mn>
  <mo mathvariant="bold">,</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
</math>

where a,b,c are real values.

We need to define a quadratic model (QM) that represents our problem. The simplest way to build a binary quadratic model (BQM) is using dimod’s symbolic variables. For each mathematical variable in your BQM, we define a symbolic
binary variable using dimod. Once the QM is defined, it is stored as a BinaryQuadraticModel object. This object stores the linear and quadratic coefficients of the mathematical expression, any constant term or offset, and
the type of variables used to build the model. In this case, printing out the object bqmmodel that we have
constructed reveals the following:

## Random not-all-equal 3-satisfiability problem (NAE3SAT)

Not-all-equal 3-satisfiability (NAE3SAT) is an NP-complete problem class that consists in satisfying a number of conjunctive clauses of three literals (variables, or their negations). For valid solutions, the literals in each clause should be not-all-equal; i.e. any assignment of values except (+1, +1, +1) or (-1, -1, -1) are valid for each clause.

Each clause contributes -1 to the energy when the clause is satisfied, and +3 when unsatisfied. The energy H(s) for a spin assignment s is thus lower bounded by E(SAT) = -num_clauses, this lower bound matches the ground state energy in satisfiable instances. The number of violated clauses is 
(H(s)-E(SAT))/4.

In [18]:
num_variables = 5;
rho = 2.1;
bqmodel = dimod.generators.random_nae3sat(num_variables, round(num_variables*rho))
print(bqmodel)


BinaryQuadraticModel({0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0}, {(1, 0): 1.0, (2, 0): -1.0, (2, 1): 0.0, (3, 0): -3.0, (3, 1): 2.0, (3, 2): 1.0, (4, 0): -3.0, (4, 1): -1.0, (4, 2): 0.0, (4, 3): 2.0}, 0.0, 'SPIN')


Define the Dynex BQM model based on our BQM:

In [19]:
bqmodel.variables

Variables([0, 1, 2, 3, 4])

In [20]:
model = dynex.BQM(bqmodel);

## Interacting with the Dynex Sampler 

To find the minimum energy state for a QM (the assignment of variable values that gives us the
minimum energy value for our QM), the Dynex SDK provides samplers and solvers. A solver is
a resource that runs a problem. Samplers are processes that run a problem many times to obtain
a collection of samples, each of which is a possible solution to our problem. For convenience, we
will generally refer to Dynex’s samplers as a whole, to include solvers as well.

In [21]:
sampler = dynex.DynexSampler(model);

Once we have established our sampler in our program, we can call it for our QM. Each type of QM
model has its own method for interacting with the sampler, whether it be QUBO, BinaryQuadrticModel, or any other QM. We call the sampler to sample our QM using one of Dynex’s sample functions, depending on what type of QM we are using. For example, the code snippet below demonstrates how we can sample a BinaryQuadraticModel object named bqm using the Dynex Platform.

In [24]:
sampleset = sampler.sample(num_reads=32, annealing_time = 100);

time: 0.00s #workers: 2 #chips: 64 #steps: 110000 global loc: 3 global energy: 4.535805
FINISHED READ AFTER 0.00 SECONDS
SAMPLESET LOADED


After we have sampled our QM, the sampler returns a SampleSet object. This object contains all
of the samples returned along with their corresponding energy value, number of chips, number of integration steps, and more. The additional information varies depending on which sampler is used. As users get more
comfortable with the Dynex SDK and the variety of samplers available, it is often useful to take
some time to explore the wealth of information provided in the SampleSet object. Some of the key properties and methods of a SampleSet that we access are the following:

In [25]:
print(sampleset[-1]) # sample with the lowest energy

{'sample': ['0.502819', '-0.985740', '0.006763', '-0.999880', '-0.528928', '0.000000', '0.000000', '-1.000000', '1.000000', '1.000000', '-1.000000', '0.000000', '0.000000', '-1.000000', '1.000000', '1.000000', '-1.000000', '-1.000000', '1.000000', '1.000000', '-1.000000', '1.000000'], 'chips': 64, 'steps': 110000, 'loc': 3, 'energy': 4.535805}


The sample shows the corresponding energy values for our QM and additional information like total energy, number of chips or number of integration steps.

In [26]:
print(sampleset[-1]['sample'][:5]) # Energy levels for the first variables:

['0.502819', '-0.985740', '0.006763', '-0.999880', '-0.528928']
